In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder \
        .appName("kafka-userdata-stream") \
        .config("spark.streaming.stopGracefullyOnShutdown", "true") \
        .getOrCreate()

In [ ]:
reguser_schema = StructType([
    StructField("username", StringType()),
    StructField("userlocation", StringType()),
    StructField("created_ts", StringType())
])

In [ ]:
kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "<KAFKA_BROKER_VM_IP:9092") \
            .option("subscribe", "userdata") \
            .option("startingOffsets", "latest") \
            .load()

In [ ]:
value_df = kafka_df.select(from_json(col("value").cast("string"), reguser_schema).alias("value"))

In [ ]:
#value_df.printSchema()

In [ ]:
exploded_df = value_df.selectExpr("value.username", "value.userlocation", "value.created_ts")

In [ ]:
reguser_writer_query = exploded_df.writeStream \
    .format("json") \
    .queryName("user writer") \
    .outputMode("append") \
    .option("path", "<GCS_BUCKET_OUTPUT>") \
    .option("checkpointLocation", "<GCS_BUCKET_CHK_POINT_DIR>) \
    .trigger(processingTime="1 minute") \
    .start()

In [ ]:
reguser_writer_query.awaitTermination()